In [5]:
import colorsys
import matplotlib.pyplot as plt
import numpy as np

def hex_to_rgb(hex_color):
    return tuple(int(hex_color[i:i+2], 16) for i in (1, 3, 5))

def rgb_to_hex(rgb):
    return f"#{rgb[0]:02x}{rgb[1]:02x}{rgb[2]:02x}"

def rgb_to_hsl(rgb):
    r, g, b = [x / 255.0 for x in rgb]
    h, l, s = colorsys.rgb_to_hls(r, g, b)
    return (h * 360, s * 100, l * 100)

def hsl_to_rgb(hsl):
    h, s, l = hsl[0] / 360.0, hsl[1] / 100.0, hsl[2] / 100.0
    r, g, b = colorsys.hls_to_rgb(h, l, s)
    return tuple(int(x * 255) for x in (r, g, b))

def get_contrast_ratio(rgb1, rgb2):
    def get_luminance(rgb):
        rgb = [x / 255.0 for x in rgb]
        rgb = [x / 12.92 if x <= 0.03928 else ((x + 0.055) / 1.055) ** 2.4 for x in rgb]
        return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]
    
    l1, l2 = get_luminance(rgb1), get_luminance(rgb2)
    return (max(l1, l2) + 0.05) / (min(l1, l2) + 0.05)

def adjust_lightness(hsl, amount):
    return (hsl[0], hsl[1], max(0, min(100, hsl[2] + amount)))

def generate_harmony_colors(base_hsl, harmony_type):
    base_hue = base_hsl[0]
    harmonies = {
        'complementary': [base_hue, (base_hue + 180) % 360],
        'analogous': [base_hue, (base_hue + 30) % 360, (base_hue - 30) % 360],
        'triadic': [base_hue, (base_hue + 120) % 360, (base_hue - 120) % 360],
        'split-complementary': [base_hue, (base_hue + 150) % 360, (base_hue - 150) % 360],
        'tetradic': [base_hue, (base_hue + 90) % 360, (base_hue + 180) % 360, (base_hue + 270) % 360],
        'monochromatic': [base_hue]
    }
    
    return [(h, base_hsl[1], base_hsl[2]) for h in harmonies[harmony_type]]

def generate_palette(colors, mode='light', harmony='complementary', bias=0):
    if isinstance(colors, str):
        colors = {'primary': colors}
    
    # Convert input colors to HSL
    hsl_colors = {k: rgb_to_hsl(hex_to_rgb(v)) for k, v in colors.items()}
    
    # Generate harmony colors if not all colors are provided
    if 'primary' in hsl_colors and len(hsl_colors) < 3:
        harmony_colors = generate_harmony_colors(hsl_colors['primary'], harmony)
        if 'secondary' not in hsl_colors:
            hsl_colors['secondary'] = harmony_colors[1]
        if 'tertiary' not in hsl_colors and len(harmony_colors) > 2:
            hsl_colors['tertiary'] = harmony_colors[2]
    
    # Apply bias
    hsl_colors = {k: ((v[0] + bias) % 360, v[1], v[2]) for k, v in hsl_colors.items()}
    
    # Generate shades
    def generate_shades(color):
        return [
            adjust_lightness(color, -30),
            color,
            adjust_lightness(color, 30)
        ]
    
    palette = {k: [rgb_to_hex(hsl_to_rgb(shade)) for shade in generate_shades(v)] 
               for k, v in hsl_colors.items()}
    
    # Generate semantic colors
    semantic_hues = {'error': 0, 'success': 120, 'info': 200, 'warning': 40}
    semantic_colors = {k: rgb_to_hex(hsl_to_rgb(((h + bias) % 360, 100, 50))) 
                       for k, h in semantic_hues.items()}
    palette['semantic'] = semantic_colors
    
    # Generate surface colors
    surface_lightness = 95 if mode == 'light' else 20
    on_surface_lightness = 10 if mode == 'light' else 90
    palette['surface'] = {
        'background': rgb_to_hex(hsl_to_rgb((0, 0, surface_lightness))),
        'onSurface': rgb_to_hex(hsl_to_rgb((0, 0, on_surface_lightness)))
    }
    
    return palette

def visualize_palette(palette):
    fig, axs = plt.subplots(len(palette), 1, figsize=(12, 3 * len(palette)))
    fig.suptitle("Color Palette")
    
    for i, (color_type, colors) in enumerate(palette.items()):
        if isinstance(colors, dict):
            axs[i].bar(range(len(colors)), [1] * len(colors), color=list(colors.values()))
            axs[i].set_xticks(range(len(colors)))
            axs[i].set_xticklabels(list(colors.keys()))
        else:
            axs[i].bar(range(len(colors)), [1] * len(colors), color=colors)
            axs[i].set_xticks(range(len(colors)))
            axs[i].set_xticklabels(colors)
        axs[i].set_title(color_type.capitalize())
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Example usage
print("Example 1: Generate palette from a single seed color")
seed_color = "#EB2116FF"  # A nice blue color
light_palette = generate_palette(seed_color, mode='light', harmony='complementary', bias=0)
dark_palette = generate_palette(seed_color, mode='dark', harmony='complementary', bias=0)

print("Light Palette:")
print(light_palette)
visualize_palette(light_palette)

print("\nDark Palette:")
print(dark_palette)
visualize_palette(dark_palette)

In [ ]:
print("\nExample 2: Generate palette from multiple input colors")
input_colors = {
    'primary': "#4287f5",
    'secondary': "#f54242",
    'tertiary': "#42f554"
}
custom_palette = generate_palette(input_colors, mode='light', harmony='analogous', bias=15)

print("Custom Palette:")
print(custom_palette)
visualize_palette(custom_palette)